# Procesamiento de datasets

En este notebook se pueden observar los procedimientos realizados a los set de datos provistos.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import spacy
nlp = spacy.load('es_core_news_sm')

#Stop Words de es_core_news_sm
from spacy.lang.es.stop_words import STOP_WORDS
stopwords_spacy = list(STOP_WORDS)

# Se quitan de las stopwords palabras como 'no', 'bueno', 'buena' que pueden ser últiles para predecir
stopwords_spacy.remove('no') 
stopwords_spacy.remove('bueno')
stopwords_spacy.remove('buena')
stopwords_spacy.remove('bien')
stopwords_spacy.remove('buenos')
stopwords_spacy.remove('tarde')
stopwords_spacy.remove('temprano')
stopwords_spacy.remove('día')
stopwords_spacy.remove('días')
stopwords_spacy.remove('dia')
stopwords_spacy.remove('dias')
stopwords_spacy.remove('grandes')
stopwords_spacy.remove('general')

In [ ]:
#Stop Words de nltk
import nltk
from nltk.corpus import stopwords
stopwords_nltk = set(stopwords.words('spanish'))

# Quito de las stopwords la palabra 'no', que nos puede ser últil para predecir
stopwords_nltk.remove('no')

import re
from nltk.tokenize import RegexpTokenizer

In [ ]:
# Importo dataset de desarrollo
data_dev = pd.read_json('dataset_amazon/dataset_es_dev.json', lines = True)
print("- Registros del set dev:", data_dev.shape[0])

# Importo dataset de entrenamiento
data_train = pd.read_json('dataset_amazon/dataset_es_train.json', lines = True)
print("- Registros del set train:", data_train.shape[0])

# Importo dataset de prueba
data_test = pd.read_json('dataset_amazon/dataset_es_test.json', lines = True)
print("- Registros del set test:", data_test.shape[0])

In [ ]:
# Concateno todos.
data = pd.concat([data_dev, data_train, data_test])
data = data.reset_index(drop=True)
data.tail()

#### Filtrado de columnas

In [ ]:
df = data[['product_category','review_title','review_body','stars']]
df.head()

#### Limpieza de datos

In [ ]:
#Constante de signos de puntuación
import string
puntua = string.punctuation + '¡¿...'
excluded_pos = ['SCONJ','CCONJ','NUM','PUNCT','PRON','DET','ADP','AUX','X','PROPN']

In [ ]:
#Función para limpieza de datos con lemmatizer
def text_data_lemma(sentence):
    doc = nlp(sentence)
    
    clean_tokens = []
    for token in doc:
        if (token.pos_ not in excluded_pos and str(token) not in stopwords_spacy and len(token.text)>2): 
            temp = token.lemma_.strip()
            clean_tokens.append(temp.lower())
    
    return clean_tokens

In [ ]:
from nltk.stem.snowball import SpanishStemmer # Permite stemmizar palabras en español
stemmer = SpanishStemmer()

#Función para limpieza de datos con stemmer
def text_data_stem(sentence):
    doc = nlp(sentence)
    
    clean_tokens = []
    for token in doc:
        if (token.pos_ not in excluded_pos and str(token) not in stopwords_spacy and len(token.text)>2): 
            temp = stemmer.stem(token.text).strip()
            clean_tokens.append(temp.lower())
    
    return clean_tokens

In [ ]:
# Lemmatización (40 min aprox)

# Limpiamos todas las reviews con lemmatizer
reviews_lemma = []
for i in df.index:
    rev = text_data_lemma(df.review_title.iloc[i] + ' ' + df.review_body.iloc[i])
    reviews_lemma.append(" ".join(rev))
reviews_lemma[:5]

In [ ]:
# Stemmización (40 min aprox)

# Limpiamos todas las reviews con stemmizer
reviews_stem = []
for i in df.index:
    rev = text_data_stem(df.review_title.iloc[i] + ' ' + df.review_body.iloc[i])
    reviews_stem.append(" ".join(rev))
reviews_stem[:5]

In [ ]:
# Guardar nuevos datos en achivo json

# Agregamos columna al dataset
df['revs_lemma'] = reviews_lemma
df['revs_stem'] = reviews_stem
df.head()

In [ ]:
# Guardamos dataset lemmatizado
df.to_json(path_or_buf='dataset_amazon_clean.json')